### Predicting Bank's Term Deposit Subscription - Preprocessing Data

#### Author: Guansu(Frances) Niu

#### Data Resource: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

In [10]:
# Imports:

import math as ma
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
from sklearn.impute import SimpleImputer
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

In [11]:
# Remove warnings:

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)

In [12]:
# Read data:

rawdata = pd.read_csv("data/raw data.csv",sep=';')
rawdata.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


#### Process Missing Values:

For continous features, due to:

‘pdays’: > 79% of data = 999, p value = 1 (MCAR test)

‘duration’: High impact on the target variable (suggested by UCL)

These two columns are dropped.

In [14]:
dropdata = rawdata.drop(columns=['pdays', 'duration'])
dropdata.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


For categorical features, the missing values are rolled into the largest category:

In [15]:
# 'job': 'unknown' is rolled into 'admin.':

# print(dropdata[rawdata['job'] == 'unknown'].shape[0]/dropdata['job'].shape[0])
# print(dropdata.job.value_counts())
jobdata = pd.DataFrame(data = dropdata['job'].replace('unknown','admin.'))
# print(jobdata.job.value_counts())

# 'marital': 'unknown' is rolled into '':

# print(dropdata[rawdata['marital'] == 'unknown'].shape[0]/dropdata['marital'].shape[0])
# print(dropdata.marital.value_counts())
maritaldata = pd.DataFrame(data = dropdata['marital'].replace('unknown','married'))
# print(maritaldata.marital.value_counts())

# 'education': 'unknown' is rolled into 'university.degree':

# print(dropdata[rawdata['education'] == 'unknown'].shape[0]/dropdata['education'].shape[0])
# print(dropdata.education.value_counts())
educationdata = pd.DataFrame(data = dropdata['education'].replace('unknown','university.degree'))
# print(educationdata.education.value_counts())

# 'default': 'unknown' is rolled into 'no':

# print(dropdata[rawdata['default'] == 'unknown'].shape[0]/dropdata['default'].shape[0])
# print(dropdata.default.value_counts())
defaultdata = pd.DataFrame(data = dropdata['default'].replace('unknown','no'))
# print(defaultdata.default.value_counts())

# 'housing': 'unknown' is rolled into 'yes':

# print(dropdata[rawdata['housing'] == 'unknown'].shape[0]/dropdata['housing'].shape[0])
# print(dropdata.housing.value_counts())
housingdata = pd.DataFrame(data = dropdata['housing'].replace('unknown','yes'))
# print(housingdata.housing.value_counts())

# 'loan': 'unknown' is rolled into 'no':

# print(dropdata[rawdata['loan'] == 'unknown'].shape[0]/dropdata['loan'].shape[0])
# print(dropdata.loan.value_counts())
loandata = pd.DataFrame(data = dropdata['loan'].replace('unknown','no'))
# print(loandata.loan.value_counts())x

In [16]:
# Create new dataset:

dropdata_1 = dropdata.drop(columns=['job', 'marital','education','default','housing','loan'])
MVframes = [jobdata,maritaldata,educationdata,defaultdata,housingdata,loandata,dropdata_1]
MVresult = pd.concat(MVframes,sort=False, axis=1)
MVresult.head()

,job,marital,education,default,housing,loan,age,contact,month,day_of_week,campaign,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,housemaid,married,basic.4y,no,no,no,56,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,services,married,high.school,no,no,no,57,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,services,married,high.school,no,yes,no,37,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,admin.,married,basic.6y,no,no,no,40,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,services,married,high.school,no,no,yes,56,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


#### Preprocess based on Data Visualization: 

In [17]:
# Remove rows that are low percentage of the features:

# 'default': remove rows have 'yes' (0.0073%):

DVdefaultresult = MVresult[MVresult['default'] != 'yes']

# 'education': remove rows have 'illiterate' (0.044%):

DVresult = DVdefaultresult[DVdefaultresult['education'] != 'illiterate']

#### Collect various features:

In [18]:
onehot_ftrs = ['job', 'marital', 'default', 'housing','loan', 'contact','poutcome']

ordinal_ftrs = ['education','month','day_of_week']

ordinal_cats = [['basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate', 'professional.course', 
                'university.degree','missing'],['mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep','oct',
                'nov','dec'],['mon', 'tue', 'wed', 'thu', 'fri']]

num_ftrs = ['age','campaign','previous','emp.var.rate','cons.price.idx','cons.conf.idx', 
                 'euribor3m','nr.employed']

#### Onehot encoder:

In [19]:
ohe = OneHotEncoder(sparse=False)
onehot_values = ohe.fit_transform(DVresult[onehot_ftrs])
onehot_ftr_names = ohe.get_feature_names()
df_onehot = pd.DataFrame(data=onehot_values,columns = onehot_ftr_names)

#### Ordinal encoder:

In [20]:
oe = OrdinalEncoder(categories = ordinal_cats)
oe_values = oe.fit_transform(DVresult[ordinal_ftrs])
df_ordinal = pd.DataFrame(data=oe_values,columns = ordinal_ftrs)

#### StandardScalar:

In [21]:
ss = StandardScaler()
num_values = ss.fit_transform(DVresult[num_ftrs])
df_num = pd.DataFrame(data=num_values,columns = num_ftrs)

#### Label encoder:

In [22]:
le = LabelEncoder()
df_le = pd.DataFrame(le.fit_transform(DVresult['y']),columns=['y'])

In [30]:
# Creating new dataframe after preprocessing:

preprocessed_frames = [df_onehot,df_ordinal,df_num,df_le]
preprocessed_data = pd.concat(preprocessed_frames,sort=False,axis=1)
preprocessed_data.to_csv('preprocessed data.csv')
preprocessed_data.head()

,x0_admin.,x0_blue-collar,x0_entrepreneur,x0_housemaid,x0_management,x0_retired,x0_self-employed,x0_services,x0_student,x0_technician,...,day_of_week,age,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.533622,-0.565901,-0.349481,0.648081,0.722531,0.886569,0.712465,0.331734,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.629594,-0.565901,-0.349481,0.648081,0.722531,0.886569,0.712465,0.331734,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,-0.289855,-0.565901,-0.349481,0.648081,0.722531,0.886569,0.712465,0.331734,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.001937,-0.565901,-0.349481,0.648081,0.722531,0.886569,0.712465,0.331734,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.533622,-0.565901,-0.349481,0.648081,0.722531,0.886569,0.712465,0.331734,0
